In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/loan-status/spaceapp-orix-data-csv-flat-normalized.csv


In [3]:
# load

ls = pd.read_csv('../input/loan-status/spaceapp-orix-data-csv-flat-normalized.csv')

In [4]:
# remove columns no, year of incorporated

ls1 = ls.drop(['No.', 'Year of Incorporated'], axis='columns')

In [6]:
# drop row that has n.a.

ls3 = ls1[~ls1.eq('n.a.').any(1)]

In [8]:
ls4 = ls3.assign(outcome=(ls3['Loan Status'] == 'GOOD').astype(int))

In [9]:
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder

cat_features = ['Industry']
encoder = LabelEncoder()

for feature in cat_features:
    e = encoder.fit_transform(ls4[feature])
    ls4[feature+'_labels'] = e

In [11]:
# drop columns Industry, Loan Status

ls5 = ls4.drop(['Industry', 'Loan Status'], axis='columns')

In [12]:
ls5.replace(',','', regex=True, inplace=True)

ls5=ls5.astype(int)

In [14]:
valid_fraction = 0.1
valid_size = int(len(ls5) * valid_fraction)

valid = ls5[:valid_size]
test = ls5[valid_size:2*valid_size]
train = ls5[2*valid_size:]

In [15]:
import lightgbm as lgb

feature_cols = train.columns.drop('outcome')

dtrain = lgb.Dataset(train[feature_cols], label=train['outcome'])
dvalid = lgb.Dataset(valid[feature_cols], label=valid['outcome'])

param = {'num_leaves': 64, 'objective': 'binary'}
param['metric'] = 'auc'
num_round = 1000
bst = lgb.train(param, dtrain, num_round, valid_sets=[dvalid], early_stopping_rounds=10, verbose_eval=False)

In [16]:
from sklearn import metrics
ypred = bst.predict(test[feature_cols])
score = metrics.roc_auc_score(test['outcome'], ypred)

print(f"Test AUC score: {score}")

Test AUC score: 0.7651515151515151


In [17]:
import pickle

# save our file (make sure our file permissions are "wb", 
# which will let us _w_rite a _b_inary file)
pickle.dump(bst, open("bst.pkl", "wb"))

In [18]:
c1 = pickle.load(open("bst.pkl", "rb"))
p = c1.predict([test[feature_cols].iloc[0]])  
print (p)

[0.89266826]


In [19]:
ypred

array([0.89266826, 0.85725327, 0.6874353 , 0.84180237, 0.94157345,
       0.96429298, 0.94311879, 0.94303523, 0.97220208, 0.91914047,
       0.84943662, 0.91673458, 0.61692048, 0.85994985, 0.88936737,
       0.87802227, 0.94521183, 0.97477788, 0.70998652, 0.67170237,
       0.97365797, 0.96846397, 0.97437942, 0.80981639, 0.93280711,
       0.85391032, 0.90716173, 0.9577763 , 0.78098574, 0.93145619,
       0.9483906 , 0.97262557, 0.91197085, 0.72302315, 0.83165591,
       0.84072582, 0.95887893, 0.82644751, 0.96081295, 0.94018467,
       0.95520548, 0.43085683, 0.97867129, 0.79240906, 0.89276975,
       0.96651924, 0.91115928, 0.95325283, 0.73606786, 0.86286982,
       0.96839628, 0.82618382, 0.33765199, 0.97488772, 0.97916203,
       0.92063491, 0.89482252, 0.90447136, 0.94192336, 0.96217869,
       0.6471468 , 0.60610907, 0.97236945, 0.97627477, 0.94147845,
       0.96214646, 0.9162746 , 0.97122766, 0.92649885, 0.93320947])

In [22]:
ypred.shape

(70,)

In [24]:
test.head()

,Paid-up Capital,Loan Amount,Loan Tenure,Non-Current Assets 2018,Non-Current Assets 2019,Current Assets 2018,Current Assets 2019,Total Assets 2018,Total Assets 2019,Current Liabilities 2018,...,Networth 2018,Networth 2019,Revenue 2018,Revenue 2019,NPBT 2018,NPBT 2019,NPAT 2018,NPAT 2019,outcome,Industry_labels
96,250001,179660,36,9717,7601,889862,892164,899579,899765,772143,...,126483,145185,5121635,5217413,21875,25060,16709,18702,0,2
97,500001,225250,60,1578,1052,804990,1434601,806568,1435653,300929,...,505639,512392,1013733,2236303,11492,10228,8554,6753,0,2
99,500001,13108,60,213232,209204,1056991,742181,1270223,951385,977514,...,222272,246207,1373270,1551058,8125,23935,8125,23935,0,0
100,500001,170000,48,134763,144305,490513,470970,625276,615275,379024,...,216963,214498,1340645,1876706,39783,-2465,30620,-2465,0,1
101,470001,13324,59,32393,402288,353198,171307,385591,573595,194400,...,189940,192574,886172,831999,18844,6823,13382,2634,1,0
